# Generate initial training set from SHARC-outputs

This script shows how to obtain an initial training set starting from calculations carried out with SHARC using Wigner sampling.

In [1]:
import ase.io
import ase
import numpy as np
import schnetpack as spk
from ase.units import Bohr
import os
SHARC="/user/julia/software/SchNarc/sharc/source/../bin/"

In [2]:
def read_QMout(data,geoms,nfiles,natoms,nnacs,ndipoles,nstates,path,soc_flag,nsinglets,ntriplets):
    dataiterator=-1
    atoms=[]
    for i in range(nfiles):
        try:
            file = open("%s_%05d/QM.out"%(path,i), "r").readlines()
            skip=False
        except IOError:
            skip=True
        if skip == False:
            dataiterator+=1
            #each data point gets one entry in the dictionary
            data[dataiterator]={}
            #each data point contains forces, so we set the following key to 1:
            data[dataiterator]["has_forces"]=np.array([1]).reshape(-1)
            
            #this iterator will tell us the line number we are in
            iterator=-1
    
            #iterate over all lines in the output
            for line in file:
                iterator+=1
                # Getting energies
                if line.startswith("! 1 Hamiltonian Matrix"):
                    energies = np.zeros((nsinglets+ntriplets,1))
                    for istate in range(nsinglets+ntriplets):
                        energyline = file[iterator+2+istate].split()
                        energies[istate]=float(energyline[2*istate])
                    data[dataiterator]["energy"]=energies.reshape(-1)
                    
                    #get spin-orbit couplings
                    if soc_flag == True:
                        #triangular minus diagonal (times two due to complex values)
                        socs = np.zeros((int(nstates*nstates-nstates)))
                        sociterator = -1
                        for istate in range(nstates*2):
                            socsline = file[iterator+2+istate].split()
                            
                            for jstate in range(2+istate*2,2*nstates):
                                sociterator+=1
                                socs[sociterator]=float(socsline[jstate])
                        data[dataiterator]["socs"] = socs
                    
                # Getting forces
                if line.startswith("! 3 Gradient Vectors "):
                    forces = np.zeros((nsinglets+ntriplets,natoms,3))
                    for istate in range(nsinglets+ntriplets):
                        for iatom in range(natoms):
                            forceline = file[iterator+2+istate+istate*natoms+iatom].split()
                            for xyz in range(3):
                                forces[istate][iatom][xyz]=float(forceline[xyz])
                    data[dataiterator]["forces"]=forces
                
                # Getting nonadiabatic couplings (NACs)
                # Only take the ones that are between different states, so in this case: skip NAC(1,1)=0
                # Note that only one NAC is saved as NAC(1,2)=-NAC(2,1) - numbers in brackets denote states
                if line.startswith("! 5 Non-adiabatic couplings"):
                    nacs = np.zeros((nnacs,natoms,3))
                    naciterator=-1
                    for inac1 in range(nstates):
                        for inac2 in range(nstates):
                            if inac1==inac2  or inac2<inac1 or (inac1 < nsinglets and inac2>=nsinglets) or inac2>=(nsinglets+ntriplets) or inac1 >=(nsinglets+ntriplets):
                                pass
                            else:
                                naciterator+=1
                                for iatom in range(natoms):
                                    nacline = file[iterator+2+inac1*nstates+inac1*natoms*nstates+inac2*natoms+inac2+iatom].split()
                                    for xyz in range(3):
                                        nacs[naciterator][iatom][xyz] = float(nacline[xyz])
                    data[dataiterator]["nacs"]=nacs
                
                # Getting values for dipole moments
                # Transition and permanent dipole moments are included in x,y,z direction (last dimension)
                # The values are read line after line starting from the diagonal values
                # In the QM.out file, there are 3 blocks, one block belongs to one direction (x,y,z)
                # SchNarc has ( ndipoles x (xyz) ) as a shape
                # Again we only take one diagonal of the matrix and only real values (imaginary values are 0)
                if line.startswith("! 2 Dipole Moment Matrices "):
                    dipoles = np.zeros((ndipoles,3))
                    for xyz in range(3):
                        dipoleiterator=-1
                        for idipole1 in range(nstates):
                            for idipole2 in range(nstates):
                                if idipole2<idipole1 or (idipole1 < (nsinglets) and idipole2>=nsinglets) or idipole2>=(nsinglets+ntriplets) or idipole1 >=(nsinglets+ntriplets):
                                    pass
                                else:
                                    dipoleiterator+=1
                                    dipoleline = file[iterator+2+xyz+xyz*nstates+idipole1].split()
                                    dipoles[dipoleiterator][xyz] = float(dipoleline[idipole2*2])
                    data[dataiterator]["dipoles"]=dipoles

            # convert the geometries into bohr
            atoms.append(ase.atoms.Atoms(geoms[i].get_atomic_numbers(),geoms[i].get_positions()/Bohr))
            
    return data, atoms
        
        

In [3]:

def read_QMout_phases(data,geoms,nfiles,natoms,nnacs,ndipoles,nstates,path, socs_flag, nsinglets, ntriplets):
    dataiterator=-1
    overlapcalc=[]
    for i in range(nfiles):
        
        try:
            file = open("%s_%05d/QM.out"%(path,i), "r").readlines()
            skip=False
        except IOError:
            skip=True
        if skip == False:
            dataiterator+=1
            #each data point gets one entry in the dictionary
            phasevector = np.ones((nsinglets+ntriplets))
    
            #this iterator will tell us the line number we are in
            iterator=-1
            found_overlap = False
            #iterate over all lines in the output
            for line in file:
                iterator+=1
                # Overlap matrices
                if line.startswith("! 6 Overlap matrix"):
                    
                    overlapiterator=-1
                    for istate in range(nsinglets+ntriplets):
                        overlapiterator+=1
                        overlapline = file[iterator+2+overlapiterator].split()
                        #skip imaginary values (all 0)
                        #this asks about the diagonal values of the overlap matrix
                        if np.abs(float(overlapline[2*istate])) >= 0.5:
                            found_overlap=True
                            if float(overlapline[2*istate]) >= 0.5:
                                phasevector[istate] = +1
                            else:
                                phasevector[istate] = -1
                        else:
                            # if the overlap are not large enough for one state, then it could be that states have switched. 
                            # we will check for this below
                            # this asks about all entries
                            for jstate in range(nsinglets+ntriplets):
                                if np.abs(float(overlapline[2*jstate])) >= 0.5:
                                    found_overlap = True
                                    if float(overlapline[2*jstate]) >= 0.5:
                                        phasevector[istate] = +1
                                    else:
                                        phasevector[istate] = -1     
                                        
                    data[dataiterator]["phases"]=phasevector

                    # if no overlap is >= 0.5
                    if found_overlap == False:
                        #print("overlap not large enough for geometry %i and state %i" %(i,istate))
                        if i not in overlapcalc:
                            overlapcalc.append(i)
                    
    dataiterator=-1
    for i in range(nfiles):
        try:
            file = open("%s_%05d/QM.out"%(path,i), "r").readlines()
            skip=False
        except IOError:
            skip=True
        if skip == False:
            dataiterator+=1
            if "phases" not in data[dataiterator]:
                overlapcalc.append(i)
            
    print("You have to interpolate for %i calculations: " %len(overlapcalc))
    print(overlapcalc)  
    return data, overlapcalc

In [4]:
def correct_phases(data,nstates,nsinglets,ntriplets,nointerpolate):
    
    corrected_data = {}
    phases=np.ones((nsinglets+ntriplets))
    dataiterator=-1
    for i in range(len(data)):
        
        # skip correction if no phase vector was found
        # only if we don't interpolate
        
        if nointerpolate==None:
            #multiply with previous phases
            phases = phases * data[i]["phases"]
            skip = False
        else:
            if i in nointerpolate:
                skip=True
            else:
                skip=False
        if skip==True:
            pass
        else:
            dataiterator+=1 
            corrected_data[dataiterator]={}
            #Energies and forces do not need to be corrected
            if "energy" in data[i]:
                corrected_data[dataiterator]["energy"] = data[i]["energy"]
                
            if "forces" in data[i]:
                corrected_data[dataiterator]["forces"] = data[i]["forces"]
                corrected_data[dataiterator]["has_forces"] = data[i]["has_forces"]
            
            #socs
            if "socs" in data[i]:
                corrected_data[dataiterator]["socs"] = data[i]["socs"]
                sociterator=-1
                
                for istate in range(nstates*2):

                    for jstate in range(2+istate*2,nstates*2):
                        if istate < (nsinglets+ntriplets): 
                            state1 = istate
                        if jstate < (nsinglets+ntriplets):
                            state2 = jstate
                        if istate >= (nsinglets+ntriplets) and istate < (nsinglets+2*ntriplets):
                            state1 = istate - ntriplets
                        if istate >= (nsinglets+ntriplets*2):
                            state1 = istate - 2*ntriplets
                        if jstate >= (nsinglets+ntriplets) and istate < (nsinglets+2*ntriplets):
                            state2 = jstate - ntriplets
                        if jstate >= (nsinglets+2*ntriplets):
                            state2 = jstate - ntriplets*2
                        sociterator+=1
                        corrected[data]["socs"][sociterator] = data[i]["socs"] * phases[state1] * phases[state2]
            # Nonadiabatic couplings
            
            if "nacs" in data[i]:
                naciterator=-1
                corrected_data[dataiterator]["nacs"] = data[i]["nacs"]
                for inac1 in range(nsinglets):
                    for inac2 in range(inac1+1,nsinglets):
                        naciterator+=1
                        corrected_data[dataiterator]["nacs"][naciterator]=data[i]["nacs"][naciterator] * phases[inac1] * phases[inac2]
                for inac1 in range(nsinglets,nsinglets+ntriplets):
                    for inac2 in range(inac1+1,nsinglets+ntriplets):
                        naciterator+=1
                        corrected_data[dataiterator]["nacs"][naciterator]=data[i]["nacs"][naciterator] * phases[inac1] * phases[inac2]
             
            # dipoles
            # only offdiagonal elements are corrected
            
            if "dipoles" in data[i]:
                dipoleiterator=-1
                corrected_data[dataiterator]["dipoles"] = data[i]["dipoles"]
                for idipole1 in range(nsinglets):
                    for idipole2 in range(nsinglets):
                        if idipole2< idipole1:
                            pass
                        else:
                            dipoleiterator+=1
                            corrected_data[dataiterator]["dipoles"][dipoleiterator]=data[i]["dipoles"][dipoleiterator] * phases[idipole1] * phases[idipole2]
                for idipole1 in range(nsinglets,nsinglets+ntriplets):
                    for idipole2 in range(nsinglets,nsinglets+ntriplets):
                        if idipole2< idipole1:
                            pass
                        else:
                            dipoleiterator+=1
                            corrected_data[dataiterator]["dipoles"][dipoleiterator]=data[i]["dipoles"][dipoleiterator] * phases[idipole1] * phases[idipole2]
                                     
                            
    return corrected_data

## Read the QM outputs and generate the training set


Note that units are all given in a.u. and that the geometries need to be saved in a.u. as well

In [5]:
#iterating over all files
nfiles = 100
#define number of states
ntriplets = 0
nsinglets = 2
nstates = nsinglets + 3 * ntriplets

#define number of atoms
natoms = 12
#number of nacs
nnacs = int(nsinglets*(nsinglets-1)/2) + int(ntriplets*(ntriplets-1)/2)
#number of dipole moment values
ndipoles = int(nsinglets+ntriplets+nnacs)

#read geometries --> these are in Angström!
geoms = ase.io.read("initconds.xyz",":")

#for conversion of atoms into bohr
from ase.units import Bohr

#data dictionary for updating the data base
data = {}

path="./ICOND"
#we don't have spin-orbit couplings
socs=False
#read properties
data,atoms=read_QMout(data,geoms,nfiles,natoms,nnacs,ndipoles,nstates,path,socs,nsinglets,ntriplets)





## Including phase correction

If you wish to include phase correction, you need to have computed the overlap matrix between your chosen starting geometry and every other geometry you want to include in the training set.
If you don't want to do phase correction, skip this section and go directly to the section "Making the data set".

### QM calculations: getting the phase vectors

you can use the function read_QMout_phases to find the phase-vectors to correct data

you will update the data dictionary and will get an array with indices that point to the folders that cannot be phase corrected without interpolation

In [6]:
data,indices_no_phases=read_QMout_phases(data,geoms,nfiles,natoms,nnacs,ndipoles,nstates,path,socs,nsinglets,ntriplets)
# the first calculation does not contain phases, as it is the reference in this case. 
# we define the phase vector as 1,1
data[0]["phases"]=np.ones((nsinglets+ntriplets))
print(len(data), "data points found.")


You have to interpolate for 98 calculations: 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 16, 17, 19, 20, 22, 23, 25, 26, 28, 29, 31, 32, 34, 35, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 0, 12, 15, 18, 21, 24, 27, 30, 33, 60, 80]
100 data points found.


### Interpolation between geometries with insufficiently large overlaps

As you can see, only a few calculations have sufficiently large overlaps with the chosen starting geometry. All these calculations require interpolation between the starting geometry and the desired geometry. An overlap calculation has to be carried out from the starting geometry to the geometry that should be included in the set.
The phases for each calculation have to be found and multiplied with the phases of the previous geometry.

You can find an example in the folder: Interpolation_Geometry1
* start.xyz: geometry of the chosen reference point
* end.xyz: geometry which should be included in the training set


#### Interpolate:

Execute "interpolate.sh" to interpolate 30 geometries between start.xyz and end.xyz. Note that the calculations cannot be done in parallel.
You will receive several files named "interpolateXX.xyz". Each file is one step in the interpolation. all geometries will be saved in a file "geometries.xyz" that can be visualized with e.g. molden. Remember the units are in Bohr.

You can then carry out each calculation serially and compute the wave function overlaps. If you wish to include these data points in the training set, then compute all desired properties for each step.

You will already be provided with the folders including the computations carried out.

We will now read in all properties and the phasevectors

In [7]:
pathinterpolate = "./Interpolation_Geometry1/Calc"

#iterating over all files
nfiles = 30

#read geometries
geoms = ase.io.read("Interpolation_Geometry1/geometries.xyz",":")

#data dictionary for updating the data base
datainterpolate = {}


datainterpolate,atomsinterpolate=read_QMout(datainterpolate,geoms,nfiles,natoms,nnacs,ndipoles,nstates,pathinterpolate,socs,nsinglets,ntriplets)
datainterpolate,indices_interpolate_no_phases=read_QMout_phases(datainterpolate,geoms,nfiles,natoms,nnacs,ndipoles,nstates,pathinterpolate,socs,nsinglets,ntriplets)


You have to interpolate for 0 calculations: 
[]


Now we need to multiply each phase vector to find the phase vector of the final geometry

In [8]:
final_phase = np.ones(nsinglets+ntriplets)
for i in range(len(datainterpolate)):
    final_phase = final_phase * datainterpolate[i]["phases"]


As you can see, the interpolation and search of the phase vector was successfull and we can now proceed to correct our data. You can find more details about the phase correction algorithm in Ref. 1. 
For all other calculations that required interpolation, we will for now assume all phasevectors have been found and are (1,1).

## Correcting the data

In this subsection, we will now use the found phasevectors to correct our data.
We will assume we have found all phasevectors (they are set to +1 for every calculation where we could not found phases for now). 

### Correct interpolated geometries

Note that for interpolated geometries we need to consecutively multiply the phase vector with every previous phase vector

In [9]:

corrected_data_interpolate = correct_phases(datainterpolate,nstates,nsinglets,ntriplets,None)

In [10]:
# correct the rest of the data for which we have phasevectors found

corrected_data = correct_phases(data,nstates,nsinglets,ntriplets,indices_no_phases)
# get relevant geometries
corrected_atoms = []
for i in range(len(atoms)):
    if i in indices_no_phases:
        pass
    else:
        corrected_atoms.append(atoms[i])


# Making the data set 

The data set will be saved in ase format (https://wiki.fysik.dtu.dk/ase/ase/db/db.html).



In [11]:
from ase.db import connect

## Without phase correction

In [18]:
# we will only write the necessary properties into the data file

keys = ["energy", "nacs", "forces", "dipoles", "has_forces", "socs"]
newdata = {}
for i in range(len(data)):
    newdata[i]={}
    for key in keys:
        if key in data:
            newdata[i][key] = data[i][key]
# define the name of the data set

dbname = "Fulvene.db"
db = connect(dbname)
for i in range(len(data)):
    #the data are in the "data" dicitonary and the atoms saved as atoms objects in the "atoms"-array
    db.write(atoms[i],data=newdata[i])
    
#define metadata
metadata = {}
metadata["info"]="Write down any information you wish to remember later, e.g., reference method, if phasecorrected, etc."

# this information is required
metadata["n_singlets"]=nsinglets
metadata["n_triplets"]=ntriplets
db.metadata=metadata
print("We have ", len(db), " data points.")

os.system(" cp %s ../../DBs/" %dbname)

We have  100  data points.


0

## Phase corrected data

In [19]:
dbname = "Fulvene_phasecorrected.db"
db = connect(dbname)

# we will only write the necessary properties into the data file

keys = ["energy", "nacs", "forces", "dipoles", "has_forces", "socs"]
newdata_corrected = {}
for i in range(len(corrected_data)):
    newdata_corrected[i]={}
    for key in keys:
        if key in corrected_data:
            newdata_corrected[i][key] = corrected_data[i][key]
# we will only write the necessary properties into the data file

newdata_corrected_interpolate = {}
for i in range(len(corrected_data_interpolate)):
    newdata_corrected_interpolate[i]={}
    for key in keys:
        if key in corrected_data_interpolate:
            newdata_corrected_interpolate[i][key] = corrected_data_interpolate[i][key]


for i in range(len(corrected_data_interpolate)):
    db.write(atomsinterpolate[i],data=newdata_corrected_interpolate[i])
#now write the other corrected data
# note that the db will not be overwritten but data will be added with the write function

for i in range(len(corrected_data)):
    db.write(corrected_atoms[i],data=newdata_corrected[i])

# again define the metadata
metadata = {}
metadata["info"]="Write down any information you wish to remember later, e.g., reference method, if phasecorrected, etc."

# this information is required
metadata["n_singlets"]=nsinglets
metadata["n_triplets"]=ntriplets
db.metadata=metadata
print("We have ", len(db), " data points.")

os.system("cp %s ../../DBs/"%dbname)

We have  32  data points.


0

Congratulations! You have generated an initial training set for SchNarc and can now use it to train your models.